In [1]:
pip install -U tensorflow_addons

     |████████████████████████████████| 679 kB 5.2 MB/s 


In [2]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir ("/content/drive/MyDrive/FourthBrain/Capstone Samsung OCT/Data/OCT2017")

Mounted at /content/drive


In [3]:
import pandas as pd
import numpy as np

In [4]:
train_dir = os.getcwd() + "/train/"
valid_dir = os.getcwd() + "/val/"
test_dir = os.getcwd() + "/test/"

In [5]:
classes = ['NORMAL', 'CNV', 'DME', 'DRUSEN']
cols = [x.lower() for x in classes]
dirs = [train_dir, valid_dir, test_dir]
label = {0: 'normal', 1: 'cnv', 2: 'dme', 3: 'drusen'}

REGEN = False # if we should read the directory structre, if False then use the CSV files already saved

In [6]:
def create_df (path, classes=classes):
  df = pd.DataFrame(columns=['filename', 'normal', 'cnv', 'dme', 'drusen'])
  for sub_dir in classes:
    condition = {'normal': 0, 'cnv': 0, 'dme':0, 'drusen': 0}
    files = os.listdir(path + sub_dir)
    if (sub_dir== 'NORMAL'):
      condition['normal'] = 1
    elif (sub_dir == 'CNV'):
      condition['cnv'] = 1
    elif (sub_dir == 'DME'):
      condition['dme'] = 1
    else:
      condition['drusen']= 1
    for f in files:
      df = df.append({'filename': path +  sub_dir  + "/" + f, 
                      'normal': condition['normal'], 
                      'cnv': condition['cnv'],
                      'dme': condition['dme'],
                      'drusen': condition['drusen']}, ignore_index=True)
  return df

In [7]:
# Generting the DataFrames of the filenames
# this is primarily used so we can sub-sample files easier for the different training strategies
if (REGEN):
  train_df = create_df(train_dir)
  valid_df = create_df(valid_dir)
  test_df = create_df(test_dir)
  train_df.to_csv("train_data.csv")
  valid_df.to_csv("valid_data.csv")
  test_df.to_csv("test_data.csv")
else:
  train_df = pd.read_csv("train_data.csv")
  valid_df = pd.read_csv("valid_data.csv")
  test_df = pd.read_csv("test_data.csv")

In [8]:
print ("Training Data: ", train_df.shape)
print ("Validation Data: ", valid_df.shape)
print ("Test Data: ", test_df.shape)

Training Data:  (83484, 6)
Validation Data:  (32, 6)
Test Data:  (968, 6)


In [9]:
# Printing out the # of samples for each subsample percentage 
print ("Trainig Data percentages:")
print (" 1% ==> ", int(.01 * train_df.shape[0]))
print (" 5% ==> ", int(.05 * train_df.shape[0]))
print ("10% ==> ", int(.1  * train_df.shape[0] ))
print ("25% ==> ", int(.25 * train_df.shape[0]))
print ("75% ==> ", int(.75 * train_df.shape[0]))
print ("90% ==> ", int(.9  * train_df.shape[0]))
print ("98% ==> ", int(.98 * train_df.shape[0]))

Trainig Data percentages:
 1% ==>  834
 5% ==>  4174
10% ==>  8348
25% ==>  20871
75% ==>  62613
90% ==>  75135
98% ==>  81814


In [10]:
# Sampling 1% of the data
sample = train_df.sample(frac=0.01, random_state=10, axis=0)
sample.shape

(835, 6)

In [11]:
# determine class weights to feed into neural network during training
def get_classweight(df):
  total = df.shape[0]
  num_norm = df['normal'].sum()
  num_cnv = df['cnv'].sum()
  num_dme = df['dme'].sum()
  num_drusen = df['drusen'].sum()
  norm_weight = (1/num_norm) * (total/4)
  norm_cnv = (1/num_cnv) * (total/4)
  norm_dme = (1/num_dme) * (total/4)
  norm_drusen = (1/num_drusen) * (total/4)
  class_weight = {0 : norm_weight, 1: norm_cnv,
                  2 : norm_dme, 3: norm_drusen}
  return class_weight

In [12]:
class_weight = get_classweight(sample)
class_weight

{0: 0.8186274509803921,
 1: 0.5537135278514589,
 2: 2.0072115384615388,
 3: 2.1085858585858586}

In [13]:
os.getcwd()

'/content/drive/My Drive/FourthBrain/Capstone Samsung OCT/Data/OCT2017'

In [14]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_hub as hub
from data_util import *
import tensorflow_addons as tfa

Total number of parameters: 23.56M


In [21]:
#gs://simclr-checkpoints/simclrv2/finetuned_100pct/r101_2x_sk1/hub
gs_path = "gs://simclr-checkpoints/simclrv2/finetuned_100pct/r50_1x_sk0/hub/"

input = tf.keras.layers.Input((224,224,3))
simclr_layer = hub.KerasLayer(gs_path, trainable=False)
output = tf.keras.layers.Dense(4, activation="softmax")

model = tf.keras.Sequential([input, 
                          simclr_layer,
                          output])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer_1 (KerasLayer)   (None, 2048)              34277672  
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 8196      
Total params: 34,285,868
Trainable params: 8,196
Non-trainable params: 34,277,672
_________________________________________________________________


In [23]:
trn_img = keras.preprocessing.image_dataset_from_directory(train_dir, labels="inferred", color_mode="rgb", batch_size=32,
                                                           image_size=(224, 224), shuffle=True, seed=10, validation_split=0.5,
                                                           subset='training', interpolation="bilinear", follow_links=False)
val_img = keras.preprocessing.image_dataset_from_directory(train_dir, labels="inferred", color_mode="rgb", batch_size=32,
                                                           image_size=(224, 224), shuffle=True, seed=10, validation_split=.05,
                                                           subset='validation', interpolation="bilinear", follow_links=False)

Found 83484 files belonging to 4 classes.
Using 41742 files for training.
Found 83484 files belonging to 4 classes.
Using 4174 files for validation.


In [24]:
model.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-3), loss='categorical_crossentropy', 
              metrics=['accuracy'])

In [25]:
# Creating a checkpoint to save the best model so that we can reload it once training is complete
checkpoint_cb = keras.callbacks.ModelCheckpoint("oct_simclrtrain.h5", save_best_only=True)
# Adding an an early stop callback to avoid overfitting in case the model is not improving after 5 consescutive epochs
earlystop_cb = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)

In [26]:
#history = model.fit(trn_img, steps_per_epoch=15, epochs=100, verbose=1, validation_data=val_img, class_weight=class_weight, callbacks=[checkpoint_cb, earlystop_cb])

In [27]:
def get_features(dataset, max_iter=100):
  out_feature, out_label = [], []
  count = 0
  for img, lbl in dataset:
    out_feature.append(simclr_layer(img).numpy())
    out_label.append(lbl.numpy())
    if count % 100 == 0: print(f'count = {count}')
    if count > max_iter:
      break
    count += 1
  out_label = np.hstack(out_label)
  out_feature   = np.vstack(out_feature)
  return out_feature, out_label

In [28]:
valid_feature, valid_label = get_features(val_img)

RuntimeError: ignored

In [ ]:
type(val_img)

In [ ]:
def get_representation(dataset, max_batch=500):
  out_rep, out_label = [], []
  count = 0
  for img, lb in dataset:
    out_rep.append(simclr_layer(img).numpy())
    out_label.append(lb.numpy())
    if count % 100 == 0: print(f'count = {count}')
    if count > max_batch: break
    count += 1
  out_label = np.hstack(out_label)
  out_rep   = np.vstack(out_rep)
  return out_rep, out_label

In [ ]:
train_rep, train_lb = get_representation(trn_img)